In [1]:
import numpy as np

# Data Preparation

In [2]:
import data_module as dm
DLS_X, DLS_Y, DLS_W, DLD_X, DLD_Y, DLD_W = dm.import_data()
#DLS = data large scale, contains thousands of datapoints
#DLD = data low dimension, contains less than or hundreds of datapoints

In [5]:
#select DLD first, as we gonna try to build up the model (faster execution)
data_w = DLD_W[0] #maximum weight
data_x = DLD_X[0] #datapoints
data_y = DLD_Y[0] #optimum values

# Solution Representation
The solution is represented as boolean mask of $N$ size, where $N$ is the size of data. 1 or True means the consecutive data is selected / carried away. 0 or False menas the consecutive data is not selected/carried away

In [6]:
sol = np.random.random(len(data_x))
sol

array([0.65583598, 0.36402171, 0.44626857, 0.50834443, 0.72500097,
       0.48874843, 0.88379197, 0.53593852, 0.91033789, 0.72171027,
       0.88801462, 0.49442212, 0.83605386, 0.20270591, 0.35578623,
       0.60825896, 0.16294387, 0.63119527, 0.87439084, 0.04075064])

In [7]:
#after discretizatoin
np.round(sol)

array([1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 0.])

In [8]:
idx = np.arange(len(data_x))
idx = idx[np.round(sol).astype(bool)]
carried = data_x.iloc[idx]
carried

,weight,price
1,91,84
4,46,4
5,55,44
7,35,82
8,75,92
9,61,25
10,15,83
11,77,56
13,63,58
16,75,70


# Fitness Function

In [11]:
def fit(sol, data, max_w) :
    #Discretization
    mask = np.round(sol).astype(bool)
    
    #decode
    idx = np.arange(len(data))
    idx = idx[mask]
    data_solution = data.iloc[idx]
    
    #price 
    price = np.sum(data_solution['price'])
    weight = np.sum(data_solution['weight'])
    if weight <= max_w:
        return price
    else :
        return 0

In [21]:
# Hyper parameters 
fmin = 0
fmax = 1
A = 1
alpha = 0.98
gamma = 0.98
population = 20

In [13]:
# def init_x (population, data_size):
#     return np.random.random((population,data_size))

# def init_f (population, fmin,fmax):
#     return  np.random.uniform(fmin,fmax,population)

# def init_v (poulation):
#     return np.zeros(population)

# def init_y (solutions,data):
#     #solutions : list of solution
#     Y = np.zeros(len(solutions))
#     for i,sol in enumerate(solutions) : 
#         Y[i] = fit(sol,data)
#     return Y

# def init_r (population):
#     return np.random.random(population)

# def init_A (population):
#     # Aku lupa A itu apa
#     return None

In [22]:
class bat:
    def __init__(self, population, data, max_w, fmin, fmax, A, alpha, gamma):
        self.population = population
        self.data = data
        self.max_w = max_w
        self.fmin = fmin
        self.fmax = fmax
        self.A = A
        self.alpha = alpha
        self.gamma = gamma
        self.data_size = len(data)
                
    def init_x (self):
        self.solutions =  np.random.random((self.population,self.data_size))
        

    def init_f (self):
        return  np.random.uniform(self.fmin,self.fmax,self.population)

    def init_v (self):
        return np.zeros(self.population)

    def init_y (self):
        
        Y = np.zeros(len(self.solutions))
        for i,sol in enumerate(self.solutions) : 
            Y[i] = fit(sol,self.data, self.max_w)
        
        self.Y = Y

    def init_r (self):
        self.r = np.random.random(self.population)

#     def init_A (self):
#         self.A_ = 

In [23]:
baat = bat(10,data_x,data_w,fmin,fmax,A,alpha,gamma)
baat.init_x()
baat.init_f()
baat.init_v()
baat.init_y()
baat.init_r()

In [27]:
len(baat.solutions[0])

20